#### Packages installation

In [1]:
!pip install "git+https://github.com/panalexeu/horchunk.git"
!pip install "git+https://github.com/panalexeu/horchunk.git"
!pip install numpy
!pip install rich

/bin/bash: line 1: pip: command not found
/bin/bash: line 1: pip: command not found
/bin/bash: line 1: pip: command not found
/bin/bash: line 1: pip: command not found


#### Define chunker wrapper to evaluate over chunking_evaluation framework

In [2]:
from horchunk.chunkers import WindowChunker 
from horchunk.splitters import SentenceSplitter 
from chromadb.utils import embedding_functions
from chunking_evaluation.chunking.base_chunker import BaseChunker

ef = embedding_functions.DefaultEmbeddingFunction() # all-MiniLM-L6-v2

# thresh: 0.61, max_chunk_size=6
class WinChunkerWrapper(BaseChunker):    
    def __init__(self, thresh: float, max_chunk_size: int):
        self.thresh = thresh
        self.max_chunk_size = max_chunk_size
        
    def split_text(self, text: str) -> list[str]:
        chunker = WindowChunker(
            ef,
            thresh=self.thresh,
            max_chunk_size=self.max_chunk_size
        )
        splits = SentenceSplitter(text).__call__()
        chunks = chunker.__call__(splits)

        return [chunk.join() for chunk in chunks]

/home/oleksii/python/semantic_chunking/semchunk/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Start evaluation

In [3]:
def parse_res(res: dict) -> dict:
    return dict(
        iou_mean=res['iou_mean'],
        iou_std=res['iou_std'],
        recall_mean=res['recall_mean'],
        recall_std=res['recall_std'],
        precision_omeaga_mean=res['precision_omega_mean'],
        precision_omeaga_std=res['precision_omega_std'],
        precision_mean=res['precision_mean'], 
        precision_std=res['precision_std']
    )

In [4]:
from chunking_evaluation import GeneralEvaluation
from rich import print

chunker = WinChunkerWrapper(thresh=0.72, max_chunk_size=3)
res = GeneralEvaluation().run(chunker, ef)
print(parse_res(res))

100%|███████████████████████████████████████| 4458/4458 [06:29<00:00, 11.44it/s]
⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.


{
    'iou_mean': np.float64(0.09959291948793635),
    'iou_std': np.float64(0.08290658960841847),
    'recall_mean': np.float64(0.6847214349978743),
    'recall_std': np.float64(0.41417763852481126),
    'precision_omeaga_mean': np.float64(0.4837208428268089),
    'precision_omeaga_std': np.float64(0.19830973720375783),
    'precision_mean': np.float64(0.10227468309034964),
    'precision_std': np.float64(0.0850018299329252)
}

In [5]:
from chunking_evaluation import GeneralEvaluation
from rich import print

chunker = WinChunkerWrapper(thresh=0.72, max_chunk_size=6)
res = GeneralEvaluation().run(chunker, ef)
print(parse_res(res))

100%|███████████████████████████████████████| 4458/4458 [06:28<00:00, 11.47it/s]


{
    'iou_mean': np.float64(0.074186023277489),
    'iou_std': np.float64(0.06341177038441934),
    'recall_mean': np.float64(0.7473691505965477),
    'recall_std': np.float64(0.40175414554324956),
    'precision_omeaga_mean': np.float64(0.35019504350976166),
    'precision_omeaga_std': np.float64(0.1831203236142467),
    'precision_mean': np.float64(0.0754785792165497),
    'precision_std': np.float64(0.06523260978001265)
}

In [6]:
from chunking_evaluation import GeneralEvaluation
from rich import print

chunker = WinChunkerWrapper(thresh=0.72, max_chunk_size=9)
res = GeneralEvaluation().run(chunker, ef)
print(parse_res(res))

100%|███████████████████████████████████████| 4458/4458 [06:28<00:00, 11.47it/s]


{
    'iou_mean': np.float64(0.061641382997508355),
    'iou_std': np.float64(0.051953356454781864),
    'recall_mean': np.float64(0.7580301237740307),
    'recall_std': np.float64(0.3944628743590051),
    'precision_omeaga_mean': np.float64(0.2825671630160264),
    'precision_omeaga_std': np.float64(0.17571330212972971),
    'precision_mean': np.float64(0.0623840910038553),
    'precision_std': np.float64(0.0530386891700107)
}